In [ ]:
# Hi this code save the model in pt and onnx format
# The pt generated are 3: normale, with trace, with script

In [ ]:
import torch


from TopicFM.src.config.default import get_cfg_defaults
from TopicFM.src.lightning_trainer.trainer import PL_Trainer
import pytorch_lightning as pl

def get_onnx(model,dummy_image0,dummy_image1,do_pruning=False):
    print(f'Moving model to device: cpu')
    with torch.no_grad():
        input_names = ["image0","image1"]
        output_names = ["mk0", "mk1", "conf","m_bids"]
        torch.onnx.export(model, (dummy_image0,dummy_image1), "model.onnx", verbose=True,do_constant_folding=True, opset_version=14, input_names=input_names,
        output_names=output_names,dynamic_axes={"image0":{0:"batch",2:"height",3:"width",},"image1":{0:"batch",2:"height",3:"width",}, "mk0":{0:"batch",1:"points"},"mk1":{0:"batch",1:"points"},"conf":{0:"batch",1:"points"},"m_bids":{0:"batch",1:"points"}})
        print("exported")


In [ ]:
#from torch.testing._internal.jit_utils import clear_class_registry
#clear_class_registry()
def get_matcher(image_type, unlock=False):
    print("hello")
    #args = parse_args()
    # init default-cfg and merge it with the main- and data-cfg
    config = get_cfg_defaults()
    #config.merge_from_file(args.data_cfg_path)
    pl.seed_everything(config.TRAINER.SEED)  # reproducibility

    # tune when testing
    threshold = None
    if threshold is not None:
        config.LOFTR.MATCH_COARSE.THR = threshold

    # lightning module

    matcher = PL_Trainer(config, pretrained_ckpt="../TopicFM/pretrained/model_best.ckpt")

    return matcher.cpu().eval()
matcher_in = get_matcher("outdoor", unlock=True).cpu().eval()
dummy_image0 = torch.randn(1, 1, 800, 800, device="cpu")
dummy_image1 = torch.randn(1, 1, 800,800, device="cpu")

with torch.no_grad():
    matcher_in.matcher(dummy_image0,dummy_image1)
    print("working")


    traced_script_module = torch.jit.trace(matcher_in.matcher,[dummy_image0,dummy_image1])
    torch.jit.save(traced_script_module , 'trace_model.pt')
    torch.jit.save(torch.jit.script(matcher_in.matcher) , 'script_model.pt')
    torch.save(matcher_in.matcher,"model.pt")

get_onnx(traced_script_module,dummy_image0,dummy_image1,do_pruning=True)

In [2]:
!onnxsim model.onnx model_simplified.onnx

Your model contains "Tile" ops or/and "ConstantOfShape" ops. Folding these ops 
can make the simplified model much larger. If it is not expected, please specify
"--no-large-tensor" (which will lose some optimization chances)
Simplifying...
Finish! Here is the difference:
┏━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃                 ┃ Original Model ┃ Simplified Model ┃
┡━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ Add             │ 96             │ 96               │
│ ArgMax          │ 1              │ 1                │
│ Cast            │ 87             │ 30               │
│ Concat          │ 60             │ 56               │
│ Constant        │ 727            │ 335              │
│ ConstantOfShape │ 29             │ 1                │
│ Conv            │ 54             │ 54               │
│ Div             │ 55             │ 55               │
│ Einsum          │ 8              │ 8                │
│ Elu             │ 4              │ 4                │
